<a href="https://www.kaggle.com/jibo16/mapping-with-plottly?scriptVersionId=87611447" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle5 as pickle # for reading pickle with a more complex way in kaggle
import plotly.express as px # importing the plotly library for mapping
import plotly.graph_objects as go # Graphical projects of the plotly library
from urllib.request import urlopen # for making requests
import json # for reading plain json


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmax-tmin-precipdatayearly201cities/city_info/city_info.csv
/kaggle/input/tmax-tmin-precipdatayearly201cities/compiled_data_on_pkl/compiled_data.pkl


## Reading data 

In [4]:
with open('/kaggle/input/tmax-tmin-precipdatayearly201cities/compiled_data_on_pkl/compiled_data.pkl', "rb") as fh:
    compiled_data = pickle.load(fh)
compiled_data = compiled_data.drop('Unnamed: 0',axis=1)
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [5]:
cities_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv')
city_info = pd.read_csv('/kaggle/input/tmax-tmin-precipdatayearly201cities/city_info/city_info.csv')
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",dtype={"fips": str})
df['countes'] = 1

## showing general data to be plotted

In [6]:
compiled_data.head()

,ID,Date,tmax,tmin,prcp,Name,Lat,Lon
0,USC00042863,1894-01-01,60.0,41.0,0.00,Reno,39.4839,-119.7711
1,USC00042863,1894-01-02,58.0,50.0,0.40,Reno,39.4839,-119.7711
2,USC00042863,1894-01-03,57.0,42.0,0.00,Reno,39.4839,-119.7711
3,USC00042863,1894-01-04,53.0,42.0,0.28,Reno,39.4839,-119.7711
4,USC00042863,1894-01-05,50.0,38.0,0.00,Reno,39.4839,-119.7711


In [7]:
city_info.head()

,Unnamed: 0,Name,ID,Lat,Lon
0,0,Lander,USW00024021,42.8153,-108.7261
1,1,Lander,USW00024018,42.8153,-108.7261
2,2,Cheyenne,USW00014897,41.1519,-104.8061
3,3,Cheyenne,USW00094973,41.1519,-104.8061
4,4,Wausau,USW00014991,44.9258,-89.6256


In [8]:
cities_data.head()

,name,pop,lat,lon
0,New York,8287238,40.730599,-73.986581
1,Los Angeles,3826423,34.053717,-118.242727
2,Chicago,2705627,41.875555,-87.624421
3,Houston,2129784,29.758938,-95.367697
4,Philadelphia,1539313,39.952335,-75.163789


## Filtering data

In [15]:
cities_quantiles = cities_data.quantile([0.15,0.24,0.25,0.49,0.50,0.74,0.75,0.94,0.95,1])
cities_data = cities_data[cities_data['pop'] > cities_quantiles['pop'][0.75]]
cities_data['log_pop'] =np.log(cities_data['pop'])
cities_data

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,name,pop,lat,lon,log_pop
0,New York,8287238,40.730599,-73.986581,15.930227
1,Los Angeles,3826423,34.053717,-118.242727,15.157441
2,Chicago,2705627,41.875555,-87.624421,14.810844
3,Houston,2129784,29.758938,-95.367697,14.571531
4,Philadelphia,1539313,39.952335,-75.163789,14.246847
5,Phoenix,1465114,33.446768,-112.075672,14.197444
6,San Antonio,1359174,29.424600,-98.495141,14.122388
7,San Diego,1321016,32.717421,-117.162771,14.093912
8,Dallas,1219399,32.776196,-96.796899,14.013869
9,San Jose,971495,37.343850,-121.883135,13.786591


# exploratory analysis

In [16]:
histo = px.histogram(cities_data['log_pop'])
histo.show()

## mapping and first maps

In [11]:


# Choosing the data to show
cities_lat = cities_data['lat']
cities_lon = cities_data['lon']
cities_name = cities_data['name']
cities_pop = cities_data['pop']


df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

limits = [(0,2),(3,9),(10,20),(21,40),(41,100),(100,3000)]
colors = ["red","crimson","green","orange","white",'yellow']
scale = 10000
fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = cities_data[lim[0]:lim[1]]
    fig = fig.add_trace(go.Scattergeo(
        lat=df_sub['lat'],
        lon=df_sub['lon'],
        text=df_sub['name'],
        marker = dict(
        size = cities_pop/10000,
        color= colors[i],
        line_color='rgb(40,40,40)',
        line_width=0.5,
        sizemode = 'area'),
        name = '{0} - {1}'.format(lim[0],lim[1])
        ))

fig.update_geos(scope='usa',lataxis_showgrid=True,landcolor = 'rgb(217, 217, 217)')

fig.show()

In [18]:
fig.write_html('binned_cities.html')